In [1]:
import boto3
import json

In [2]:
boto3.Session().region_name

'eu-west-1'

## Lambda

In [3]:
lambda_client = boto3.client('lambda')

In [11]:
[_['FunctionName'] for _ in lambda_client.list_functions()['Functions']]

['checklist-service-dev-addCollaborator',
 'checklist-service-dev-addEntry',
 'email-service-dev-sendEmail',
 'api-gateway-vpc-dev-main',
 'slic-watch-WatchExistingResources-Q3DNIDST8STN',
 'checklist-service-dev-update',
 'sharing-service-dev-create',
 'slic-watch-DeleteResources-NSK3LQ6YLF1N',
 'api-gateway-vpc-caller-dev-caller',
 'checklist-service-dev-updateEntry',
 'welcome-service-dev-handleNewChecklist',
 'internal-api-dev-main',
 'checklist-service-dev-get',
 'checklist-service-dev-list',
 'checklist-service-dev-listEntries',
 'checklist-service-dev-delete',
 'slic-watch-LambdaInvocati-LambdaInvocationCustomRe-1E6Y83WSAEXR4',
 'checklist-service-dev-deleteEntry',
 'internal-api-caller-dev-caller',
 'api-gateway-vpc-dev-custom-resource-apigw-cw-role',
 'user-service-dev-get',
 'cfn-lex-bot-1-0-3',
 'internal-api-dev-custom-resource-apigw-cw-role',
 'user-service-dev-custom-resource-apigw-cw-role',
 'sharing-service-dev-confirm',
 'checklist-service-dev-create',
 'cfn-lex-slot-t

In [8]:
def get_applicable_lambdas():
    tags_per_func = {}
    for response in lambda_client.get_paginator('list_functions').paginate():
        for function in response['Functions']:
            name = function['FunctionName']
            func_response = lambda_client.get_function(FunctionName=name)
            func = {k:v for k, v in func_response['Configuration'].items() if k in ('FunctionName', 'FunctionArn', 'Runtime', 'Timeout', 'MemorySize')}
            print(func)
            break
            tags = func_response.get('Tags', {})
            tags_per_func[name] = tags
    return tags_per_func

In [9]:
lambda_functions = get_applicable_lambdas()

{'FunctionName': 'checklist-service-dev-addCollaborator', 'FunctionArn': 'arn:aws:lambda:eu-west-1:935672627075:function:checklist-service-dev-addCollaborator', 'Runtime': 'nodejs10.x', 'Timeout': 6, 'MemorySize': 1024}


# SQS

In [ ]:
sqs_client = boto3.client('sqs')

In [ ]:
def get_applicable_queues():
    tags_per_queue = {}
    for response in sqs_client.get_paginator('list_queues').paginate():
        for queue in response['QueueUrls']:
            tags = sqs_client.list_queue_tags(QueueUrl=queue).get('Tags', {})
            tags_per_queue[queue] = tags
    return tags_per_queue

In [ ]:
sqs_queues = get_applicable_queues()

# DynamoDB

In [ ]:
dynamo_client = boto3.client('dynamodb')

In [ ]:
tags_per_table = {}
for response in dynamo_client.get_paginator('list_tables').paginate():
    for table in response['TableNames']:
        arn = dynamo_client.describe_table(TableName=table)['Table']['TableArn']
        tags = dynamo_client.list_tags_of_resource(ResourceArn=arn)['Tags']
        tags_per_table[table] = tags
        
print(tags_per_table)

# Kinesis

In [ ]:
kinesis_client = boto3.client('kinesis')

In [ ]:
tags_per_stream = {}
for response in kinesis_client.get_paginator('list_streams').paginate():
    for stream_name in response['StreamNames']:
        stream_tags = kinesis_client.list_tags_for_stream(StreamName=stream_name)['Tags']
        tags_per_stream[stream_name] = stream_tags

In [ ]:
tags_per_stream

# Create dashboard

In [ ]:
cw_client = boto3.client('cloudwatch')

In [ ]:
METRIC_PERIOD = 60
REGION = boto3.DEFAULT_SESSION.region_name
WIDGET_WIDTH = 24
WIDGET_HEIGHT = 6
MAX_WIDTH = 24
DASHBOARD_PERIOD = '-PT3H'

In [ ]:
def create_metric_widget(title, metrics):
    return dict(
        type='metric',
        properties=dict(
            metrics=metrics,
            view='timeSeries',
            region=REGION,
            stat='Maximum',
            period=METRIC_PERIOD,
            title=title
        )        
    )

In [ ]:
def create_all_lambda_widget():
    all_lambda_metrics = [
        ['AWS/Lambda', 'Duration', {'stat': 'Maximum'}],
        ['.', '.', {'stat': 'Average'}],
        ['.', '.', {'stat': 'p95'}],
        ['.', '.', {'stat': 'Minimum'}],
        ['.', 'ConcurrentExecutions', {'stat': 'Maximum', 'yAxis': 'right'}]
    ]
    return create_metric_widget(title='All Functions Duration/Concurrent Executions', metrics=all_lambda_metrics)

In [ ]:
from collections import namedtuple

In [ ]:
Metric = namedtuple('Metric', ['metric', 'stat'])

In [ ]:
func_metrics = [
    Metric('Duration', 'Average'),
    Metric('Duration', 'p95'),
    Metric('Duration', 'Maximum'),
    Metric('Invocations', 'Sum'),
    Metric('ConcurrentExecutions', 'Maximum')
]

In [ ]:
def create_func_metrics(functions, metric):
    return [
        ['AWS/Lambda', metric.metric, 'FunctionName', func, {'stat': metric.stat }]
        for func in functions
    ]

In [ ]:
func_widgets = []
def create_function_widgets():
    return [
        create_metric_widget(
            title=f'{metric.metric} {metric.stat} per Function', 
            metrics=create_func_metrics(lambda_functions, metric) 
        ) for metric in func_metrics
    ]

In [ ]:
widgets = [create_all_lambda_widget()] + create_function_widgets()

In [ ]:
x = 0
y = 0

In [ ]:
for widget in widgets:
    if x + WIDGET_WIDTH > MAX_WIDTH:
        y += WIDGET_HEIGHT
        x = 0
    widget['x'] = x
    widget['y'] = y
    widget['width'] = WIDGET_WIDTH
    widget['height'] = WIDGET_HEIGHT
    x += WIDGET_WIDTH

In [ ]:
dash = {
    'start': DASHBOARD_PERIOD,
    'widgets': widgets
}
dash_body = json.dumps(dash)

In [ ]:
dash_response = cw_client.put_dashboard(DashboardName='SLICWatch', DashboardBody=dash_body)
print(dash_response)